Import Dependencies

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
from tensorflow.keras.datasets import mnist

Loading MNIST Dataset

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [12]:
x_train = x_train/255.0
x_test = x_test/255.0

In [13]:
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)

In [15]:
input_shape = x_train[0].shape

Setting up the Model

In [18]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size = (3,3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)              

In [21]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Training the Model

In [22]:
history = model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 58s 962us/sample - loss: 0.2419 - accuracy: 0.9269 - val_loss: 0.0505 - val_accuracy: 0.9836
Epoch 2/10
60000/60000 [==============================] - 59s 987us/sample - loss: 0.0843 - accuracy: 0.9754 - val_loss: 0.0350 - val_accuracy: 0.9883
Epoch 3/10
60000/60000 [==============================] - 59s 985us/sample - loss: 0.0631 - accuracy: 0.9802 - val_loss: 0.0310 - val_accuracy: 0.9898
Epoch 4/10
60000/60000 [==============================] - 59s 990us/sample - loss: 0.0524 - accuracy: 0.9840 - val_loss: 0.0319 - val_accuracy: 0.9893
Epoch 5/10
60000/60000 [==============================] - 61s 1ms/sample - loss: 0.0449 - accuracy: 0.9854 - val_loss: 0.0297 - val_accuracy: 0.9906
Epoch 6/10
60000/60000 [==============================] - 62s 1ms/sample - loss: 0.0375 - accuracy: 0.9878 - val_loss: 0.0273 - val_accuracy: 0.9905
Epoch 7/10
60000/60000 [========================

In [23]:
model.evaluate(x_test, y_test)

[0.025093375344562536, 0.9926]

Saving the Model

In [24]:
model.save('trainedModel.h5')